In [ ]:
from bs4 import BeautifulSoup
import lxml
import re
import os.path
my_path = os.path.abspath('')

In [ ]:
#Data cleaning approach:

#First identify the table of content by regex to get mention of "Item" in <Table> <tbody> tag(perhaps extract the table of content)
#or search <a href="#....." util the mention of Item16, signautre
#extract all the href in <a> attribute in the table of content (ex. href1, href2, href3 …)

#Second have algorithm to identify <a name="href1"> and save the content between href1 and href2, href2 and href3, …

#Third, clean the html tags in the contents 

In [ ]:
def load_html(path):
    return BeautifulSoup(open(path,"r"), 'html.parser')
def load_xml(path):
    return BeautifulSoup(open(path,"r"), 'lxml')

#get the text from soup
def soup_get_text(soup):
    print(soup.get_text())

#write "string" to file from "path"
def write_file(path, string):
    text_file = open(path, "w")
    text_file.write(string)
    text_file.close()


#print elements from list
def print_list(lists):
    for list in lists:
        print(list)

#read all the text file in the directory
def file_reader(path):
    files = []
    for r,d,f in os.walk(path):
        for file in f:
            if '.txt' in file:
                files.append(os.path.join(r,file))
                
    return files


In [ ]:
#First extract the location of the table of content by
#using regex to identify "Item 15" or "Financial Statement Schedules"
#get every links <a href="#"> above "Item 15" or "Financial Statement Schedules"
#only works with the text and items in table of content contains href#
#if only in the page number, doesn't work
def get_table_of_content_links(soup):
    tag = soup.find('a',string=re.compile("Financial Statement Schedules"))
    tag_item = soup.find('a',string=re.compile("Item 15"))
    if tag==None and tag_item!=None:
        tag = tag_item
    
    links = list()
    links.append(tag.get('href')[1:])
    for elements in tag.previous_elements:
        if elements.name=='a' and elements.has_attr('href') and elements.get('href')[0]=='#':
            #print(elements)
            links.append(elements.get('href')[1:])
    
    links.reverse()
    print("total section: ",len(remove_duplicates_in_list(links)))
    return remove_duplicates_in_list(links)

#remove dupicated items
def remove_duplicates_in_list(List):
    return list(dict.fromkeys(List))
"""
def get_table_of_content_links2(soup):
    links = list()
    for link in soup.find_all('a'):
    #link.get('href')
    #print(link.get('href'))
    #print(link.get('name'))
        if link.get('href')!=None and link.get('href')[0]=='#':
            links.append(link.get('href')[1:])
    return links
# or identify the signature link and use find_previous method to extract:extract()?
"""


In [ ]:
#get every sections by using the links to identify <a>
def get_all_sections(links, soup):
    sections=list()
    for i in range(len(links)):
        if i<len(links)-1:
            sections.append(get_section(links[i],links[i+1],soup))
        else:
            sections.append(get_last_section(links[i],soup))
    return sections

#get sections between links[i] and links[i+1]
def get_section(link1, link2, soup):
    section_str=""
    for link in soup.find_all('a'):
        if link.has_attr('name') and link['name']==link1:
            for elements in link.next_elements:
                if elements.name=='a' and elements.has_attr('name') and elements['name']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
        elif link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if elements.name=='a' and elements.has_attr('id') and elements['id']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
    return section_str

#get last section that is between <a name="links[n]"> to the end of the document
def get_last_section(link1,soup):
    section_str=""
    for link in soup.find_all('a'):
        if link.has_attr('name') and link['name']==link1:
            for elements in link.next_elements:
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
        elif link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
    return section_str

In [ ]:
#Remove all the tag that is not <type>10-k using xml
def clean_10k_by_xml(path):
    soup=load_xml(path)
    extract_type_10k(soup)
    write_file(path, str(soup))

#remove all tag that is not <type>10-k
def extract_type_10k(soup):
    tag_list=soup.find_all('type')
    for tag in tag_list:
        print(tag.next_element)
        if '10-K' not in tag.next_element:
            tag.extract()
    

In [ ]:
#First remove all the tag that is not <type>10-k including xml
#Secont extract the links of the items from the table of content 
#Third get all the sections and store it in a list
def get_10k_sections(path):
    soup=load_html(path)
    sections=get_all_sections(get_table_of_content_links(load_html(path)),soup)
    return sections

In [ ]:
#get all the path of 10k file and store in list name files
def get_10k_files():
    files = [];
    files = file_reader(my_path+"/sec_edgar_filings")
    return files

In [ ]:
files=list()
files=get_10k_files()
print_list(files)


In [ ]:
for file in files:
    print(file)
    clean_10k_by_xml(file)


In [ ]:
sections=list()
sections=get_10k_sections(files[29])


In [ ]:
print(sections[-1])

In [ ]:
#BLK file 20 link in <p id=" "
#visa file 2015 have link from item 1-14 but probably forgot to add link to < a item 15 or < a Financial Statement Schedules
#visa file 2015 add link to the page

In [ ]:
#remove all xml tag and save it 
"""
remove_xml(soup1)
text_file = open("sec_edgar_filings/clean_data/sample_10k.txt", "w")
text_file.write(str(soup1))
text_file.close()
"""
#load html soup and prettify
"""
soup=load_html("sec_edgar_filings/clean_data/sample_10k.txt")
soup.prettify()
"""
#get table of content links
"""
links = get_table_of_content_links1(soup)
print_list(links)
print(len(links))
"""
#get all sections in 10k
"""
sections=list()
get_all_sections(sections,links)
print(sections[21])
"""
#write sections to a txt file
"""
text_file = open("sample_KG.txt", "w")
text_file.write(sections[2])
text_file.close()
"""

In [ ]:
"""
MSFT_1_path='sec_edgar_filings/MSFT/10-K/0001193125-15-272806.txt'
MSFT_2_path="sec_edgar_filings/MSFT/10-K/0001564590-18-019062.txt"
APPL_1_path='sec_edgar_filings/AAPL/10-K/0000320193-19-000119.txt'
APPL_2_path='sec_edgar_filings/AAPL/10-K/0000320193-18-000145.txt'
VISA_1_path='sec_edgar_filings/V/10-K/0001403161-18-000055.txt'
VISA_2_path='sec_edgar_filings/V/10-K/0001403161-19-000050.txt'
GOOG_1_path='sec_edgar_filings/GOOG/10-K/0001652044-19-000004.txt'
GOOG_2_path='sec_edgar_filings/GOOG/10-K/0001652044-20-000008.txt'
BLK_1_path='sec_edgar_filings/BLK/10-K/0001564590-19-005479.txt'
BLK_2_path='sec_edgar_filings/BLK/10-K/0001564590-20-007807.txt'
BA_1_path='sec_edgar_filings/BA/10-K/0000012927-19-000010.txt'
BA_2_path='sec_edgar_filings/BA/10-K/0000012927-20-000014.txt'
FORD_1_path='sec_edgar_filings/F/10-K/0000037996-20-000010.txt'
FORD_2_path='sec_edgar_filings/F/10-K/0000037996-19-000012.txt'

soup=load_html(MSFT_1_path)
"""
#soup_get_text(soup)
#visa, msft, goog, blk work
#, ba_1_path uses <a id="links">
#ford dont work cuz href is at the page number in toc


In [ ]:
"""
#get table of content

"identify the mention of Signatures"
tag = soup.find('a',string=re.compile("Signatures"))
#print(tag)
links = list()
for elements in tag.previous_elements:
    if elements.name=='a' and elements.has_attr('href') and elements.get('href')[0]=='#':
        #print(elements)
        links.append(elements.get('href')[1:])
        
links.reverse()
#print_list(links)
"""

In [ ]:
#load the html
"""
soup = BeautifulSoup(open("/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/MSFT/10-K/0001564590-19-027952.txt"), 'html.parser')
#print(soup.prettify())
print(soup.get_text())
"""



In [ ]:
"""
#get all href from the table of content 
Links = list()
for link in soup.find_all('a'):
    #link.get('href')
    #print(link.get('href'))
    #print(link.get('name'))
    if link.get('href')!=None and link.get('href')[0]=='#':
        Links.append(link.get('href')[1:])
for link in Links:
    print(link)
        
    #remove the '#' and store them in a list
"""

In [ ]:
#get section 
#example get section between Item 1 and Item 1A

#Problem also get the sentences
"""
Sections=""
for link in soup.find_all('a'):
    if link.has_attr('name') and link['name']=='ITEM_1_BUSINESS':
        for elements in link.next_elements:
            if elements.name=='a' and elements.has_attr('name') and elements['name']=='ITEM_1B_UNRESOLVED_STAFF_COMMENTS':
                break
                
            
            if str(elements)[0] != '<':
                Sections = Sections + str(elements)
                print(repr(elements))
            #Sections = Sections + str(elements)
            
        break
"""
